## About this notebook

We are going to use SageMaker BYOS mode for the training and deployment part of our model.

待processing完成后，在上一个notebook中设定的output_data这个S3 路径下面将有生成的用于训练的数据。

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

_role = get_execution_role()
print(_role)

将bucket设为自己的桶名，inputs为在processing阶段设置的output_data这个值，形式为 's3://{}/spoken/processing-folds/'.format(bucket)，即处理后的将用于训练的数据

In [ ]:
bucket = 'YOUR_BUCKET_NAME'
inputs = 's3://{}/spoken/processing-folds/'.format(bucket)

prefix = 'spoken'

In [ ]:
from sagemaker.tensorflow import TensorFlow

# 建议使用gpu类型的实例
instance_type='ml.p3.2xlarge'

# 可以修改epoch_count，batch_size
sli_estimator = TensorFlow(entry_point='model.py',
                             source_dir='./source',
                             role=_role,
                             output_path='s3://{}/{}/output'.format(bucket, prefix),
                             framework_version='1.15.2',
                             hyperparameters={'epoch_count':100, 'batch_size':8}, 
                             py_version='py3',
                             train_instance_count=1,
                             train_instance_type=instance_type,
                             train_volume_size=100,
                             train_max_run=432000,
                             train_use_spot_instances=True,
                             train_max_wait=432000,
                             metric_definitions=[{'Name': 'loss', 'Regex': 'loss = (.*?),'},
                                                 {'Name':'epoch','Regex': 'Step_Train = (.*?),'}]
                           )

In [ ]:
# SageMaker will automatically download the data and make it avaliable at /opt/ml/input/data/train/
result = sli_estimator.fit({'train':inputs})

模型训练结束后，可以在S3中看到模型文件